### SPLIT AFRICA DATASET

I re-ran this notebook until the split was reasonably even between train/validation/test. In a perfect split the `low-med-high rfracs` would all equal 1 and the `block split fracs` would equal the respective `group fracs`.  

RESULTS:

```
-------------------------------------------------------------------------------------
block relative split: 0.7012195121951219 0.1991869918699187 0.09959349593495935 0.2
block split fracs: 0.5609756097560976 0.15934959349593497 0.07967479674796749 0.2
-------------------------------------------------------------------------------------

train group frac: 0.5678053830227743
* low-med-high rfracs: 0.9962155741554101 1.0129254245683184 0.9250573188585951

valid group frac: 0.16614906832298137
* low-med-high rfracs: 1.1247049938638722 0.9197639241532062 1.2665596777746309

test group frac: 0.06159420289855073
* low-med-high rfracs: 0.9839572192513368 1.0029751785107106 0.9292929292929293

hold_out group frac: 0.20445134575569357
* low-med-high rfracs: 0.9140007671653241 1.0284115021584839 1.0128116609129267

```

NOTES:

    1. validation is a little low in group count but that speeds up training. its the test group that matters
    2. the low-med-high fracs are good. the valid group is high on low water content and low on high water content which you can argue is a feature since it requires us to score better on the harder categories.
    3. the hold out group can be used for additional training or additional validation if needed.
 

---

In [1]:
import pandas as pd
from random import sample

In [2]:
DSET='surface-water.africa'
NO_DATA_MAX=0.05

In [3]:
df=pd.read_csv(f'{DSET}.csv')
print(
    df.shape[0],
    df.drop_duplicates(subset=['block_id']).shape[0],
    df.drop_duplicates(subset=['group_id']).shape[0])
df=df[df.no_data<=NO_DATA_MAX]
print(
    df.shape[0],
    df.drop_duplicates(subset=['block_id']).shape[0],
    df.drop_duplicates(subset=['group_id']).shape[0])
df.sample(3)

4517 656 4405
3956 615 3864


,gsw_path,s1_path,block_id,group_id,water,not_water,no_data,dataset
2854,gs://surface-water-public/data/v1/jrc/africa_m...,gs://surface-water-public/data/v1/sentinel_1/a...,block_27_-27,group_27.26_-27.65,0.053967,0.946033,0.000000,africa_mtn
348,gs://surface-water-public/data/v1/jrc/africa/G...,gs://surface-water-public/data/v1/sentinel_1/a...,block_5_9,group_5.8_9.02,0.076931,0.923069,0.000000,africa
3879,gs://surface-water-public/data/v1/jrc/africa_m...,gs://surface-water-public/data/v1/sentinel_1/a...,block_25_-30,group_25.9_-30.51,0.015156,0.984615,0.000229,africa_mtn


---

In [4]:
def water_counter(df,quantiles=[0.15,0.85],labels=['low','medium','high']):
    wqs=[0]+[df.water.quantile(q) for q in quantiles]+[1+1e-8]
    dfs=[]
    for i,l in enumerate(labels):
        _df=df.copy()[(df.water>=wqs[i])&(df.water<wqs[i+1])]
        _df['water_tag']=l
        dfs.append(_df)
    return pd.concat(dfs).sample(frac=1)


def split_groups(blocks,valid_frac=0.2,test_frac=0.1,hold_out_frac=0.2,relative_frac=True):
    if relative_frac:
        valid_frac=(1-hold_out_frac)*valid_frac
        test_frac=(1-hold_out_frac)*test_frac
    nb_blocks=len(blocks)
    nb_valid=int(valid_frac*nb_blocks)    
    nb_test=int(test_frac*nb_blocks)
    nb_hold_out=int(hold_out_frac*nb_blocks)
    blocks=sample(blocks,nb_blocks)
    valid=blocks[:nb_valid]
    test=blocks[nb_valid:nb_valid+nb_test]
    hold_out=blocks[nb_valid+nb_test:nb_valid+nb_test+nb_hold_out]
    train=blocks[nb_valid+nb_test+nb_hold_out:]
    rtotal=len(train)+len(valid)+len(test)
    print('block relative split:',len(train)/rtotal,len(valid)/rtotal,len(test)/rtotal,len(hold_out)/nb_blocks)
    print('block split fracs:',len(train)/nb_blocks,len(valid)/nb_blocks,len(test)/nb_blocks,len(hold_out)/nb_blocks)
    return train, valid, test, hold_out


def sub_df(typ,df,blocks,cnt):
    
    sdf=df.copy()[df.block_id.isin(blocks)]
    sdf['data_split']=typ
    
    sgcnt=sdf.drop_duplicates(subset=['group_id']).shape[0]
    frac=sgcnt/cnt
    
    ltcnt=df[df.water_tag=='low'].shape[0]*frac
    mtcnt=df[df.water_tag=='medium'].shape[0]*frac
    htcnt=df[df.water_tag=='high'].shape[0]*frac
    
    sltcnt=sdf[sdf.water_tag=='low'].shape[0]
    smtcnt=sdf[sdf.water_tag=='medium'].shape[0]
    shtcnt=sdf[sdf.water_tag=='high'].shape[0]
    print()
    print(f'{typ} group frac:',frac)
    print('* low-med-high rfracs:',sltcnt/ltcnt,smtcnt/mtcnt,shtcnt/htcnt)
    return sdf
    
    
def df_splitter(df,train, valid, test, hold_out):
    cnt=df.drop_duplicates(subset=['group_id']).shape[0]
    tdf=sub_df('train',df,train,cnt)    
    vdf=sub_df('valid',df,valid,cnt)    
    sdf=sub_df('test',df,test,cnt)    
    hdf=sub_df('hold_out',df,hold_out,cnt)
    df=pd.concat([tdf, vdf, sdf, hdf])
    return df

---

In [5]:
df=water_counter(df)

In [6]:
blocks=list(df.block_id.unique())

In [7]:
print('-'*85)
train, valid, test, hold_out=split_groups(blocks)
print('-'*85)
df=df_splitter(df, train, valid, test, hold_out)

----------------------------------------------------------------------------------------------------
block relative split: 0.7012195121951219 0.1991869918699187 0.09959349593495935 0.2
block split fracs: 0.5609756097560976 0.15934959349593497 0.07967479674796749 0.2
----------------------------------------------------------------------------------------------------

train group frac: 0.5678053830227743
* low-med-high rfracs: 0.9962155741554101 1.0129254245683184 0.9250573188585951

valid group frac: 0.16614906832298137
* low-med-high rfracs: 1.1247049938638722 0.9197639241532062 1.2665596777746309

test group frac: 0.06159420289855073
* low-med-high rfracs: 0.9839572192513368 1.0029751785107106 0.9292929292929293

hold_out group frac: 0.20445134575569357
* low-med-high rfracs: 0.9140007671653241 1.0284115021584839 1.0128116609129267


---

In [8]:
df.sample(3)

,gsw_path,s1_path,block_id,group_id,water,not_water,no_data,dataset,water_tag,data_split
3230,gs://surface-water-public/data/v1/jrc/africa_m...,gs://surface-water-public/data/v1/sentinel_1/a...,block_28_-32,group_28.56_-32.01,0.018238,0.959488,0.022274,africa_mtn,medium,train
1466,gs://surface-water-public/data/v1/jrc/africa/G...,gs://surface-water-public/data/v1/sentinel_1/a...,block_30_28,group_30.77_28.34,0.076309,0.920403,0.003288,africa,medium,train
2846,gs://surface-water-public/data/v1/jrc/africa_m...,gs://surface-water-public/data/v1/sentinel_1/a...,block_27_-31,group_27.64_-31.88,0.066280,0.933720,0.000000,africa_mtn,medium,train


In [9]:
file_name=f'{DSET}.split.csv'
df.to_csv(file_name,index=False)
print(f'UPLOAD: gsutil cp {file_name} gs://surface-water-public/data/v1/datasets')

UPLOAD: gsutil cp surface-water.africa.split.csv gs://surface-water-public/data/v1/datasets


---

In [16]:
# !gsutil cp surface-water.africa.split.csv gs://surface-water-public/data/v1/datasets

Copying file://surface-water.africa.split.csv [Content-Type=text/csv]...
/ [1 files][  1.2 MiB/  1.2 MiB]                                                
Operation completed over 1 objects/1.2 MiB.                                      


---

##### WATER/NOT RATIO

In [11]:
_df=df[df.data_split.isin(['train','valid'])]

In [12]:
_df.not_water.mean()/_df.water.mean()

12.218997545877423

In [13]:
_df.drop_duplicates(subset=['group_id'])[['water','not_water']].describe()

,water,not_water
count,2836.000000,2836.000000
mean,0.075401,0.919694
std,0.083581,0.085145
min,0.005032,0.411617
25%,0.017901,0.893487
50%,0.038324,0.956427
75%,0.100456,0.978686
max,0.588383,0.994911
